In [7]:
#coding=utf-8
import numpy as np
import pandas as pd
import random
import time
import json
import sys
import gc
from tqdm import tqdm
import matplotlib.pyplot as plt
# machine learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, roc_auc_score
from collections import Counter
from sklearn.model_selection import GroupKFold
import xgboost
import lightgbm as lgb

data_path = r'D:\kaggle\data\tianchi_disk\a'[: -1]

def read_data_csv(data_name, mark = -1):
    if mark < 0:
        df = pd.read_csv(data_path + data_name + '.csv')
    else:
        df = pd.read_csv(data_path + data_name + '.csv', nrows = mark)
    return df

In [8]:
def take_sample():
    read_rows = int(sys.argv[1])
    data_pre_name = 'disk_sample_smart_log_'
    df_label = pd.read_csv(data_path + 'disk_sample_fault_tag.csv', names = ['manufacturer', 'model', 'serial_number', 'fault_time', 'tag'])
    df_label = df_label.groupby(['manufacturer', 'model', 'serial_number'])['fault_time'].min().to_frame().reset_index()
    print df_label.head(3)
    for col in ['manufacturer', 'model', 'serial_number']: df_label[col] = df_label[col].astype(str)
    for day in range(201707, 201713) + range(201801, 201808):
        # if day <= 201712: continue
        df = read_data_csv(data_pre_name + str(day), read_rows)
        print 'raw : ', len(df)
        cnt = df.shape
        for col in ['manufacturer', 'model', 'serial_number']: df[col] = df[col].astype(str)
        df = df.merge(df_label, on = ['manufacturer', 'model', 'serial_number'], how = 'left')
        df_positive = df[~df['fault_time'].isna()]
        df = df[df['fault_time'].isna()]
        df = df.sample(n = len(df_positive) * 10, random_state = 2020)
        df = pd.concat([df_positive, df])
        df.to_csv(data_path + data_pre_name + str(day) + '_sample_pn_v2.csv', index = False)
        print 'sample : ', day, cnt, len(df_positive), len(df)
        del df, df_positive
        gc.collect()
    return df


In [4]:
df_label = pd.read_csv(data_path + 'disk_sample_fault_tag.csv', names = ['manufacturer', 'model', 'serial_number', 'fault_time', 'tag'])
df_label = df_label.groupby(['manufacturer', 'model', 'serial_number'])['fault_time'].min().to_frame().reset_index()


In [15]:
def dropna_ratio(day):
    data_pre_name = 'disk_sample_smart_log_'
    data_name = data_pre_name + str(day) + '.csv'
    df = pd.read_csv(data_path + data_name).sample(frac = 0.2, random_state = 2020)
    cnt = len(df)
    x = df.isna()
    del df
    gc.collect()
    x = x.sum() * 1.0 / cnt
    plt.hist(x = x, bins = 20)
    plt.title(str(day))
    plt.show()

    del x
    gc.collect()
for day in range(201707, 201713) + range(201801, 201808):
    dropna_ratio(201707)

MemoryError: 